# 🏋️ Azure AI Search + Semantic Kernel + AI Agents: Fitness-Fun Workshop 🤸

Welcome to this self-guided workshop where you'll:

1. **Create** an Azure AI Search index containing some sample fitness equipment data
2. **Upload** and verify your documents
3. **Create** a Semantic Kernel Agent (powered by Azure AI Agent Service) that uses the Azure AI Search tool
4. **Run** an asynchronous conversation to query your index (with a fun fitness twist)

> **Note:** This demo uses Semantic Kernel’s abstractions over Azure AI Agents. Make sure you run:
>
> ```bash
> pip install semantic-kernel[azure]
> ```

Also ensure you’ve set your environment variables for:

- `PROJECT_CONNECTION_STRING`
- `MODEL_DEPLOYMENT_NAME`

Let’s get started!

## Prerequisites

Before running the cells below, please verify:

1. You have installed the required dependency:
   ```bash
   pip install semantic-kernel[azure]
   ```
2. Your environment is configured with the necessary variables (`PROJECT_CONNECTION_STRING` and `MODEL_DEPLOYMENT_NAME`).

In [7]:
# Uncomment and run the cell below if you have not installed the dependency yet
# !pip install semantic-kernel[azure]

## 1. Create & Populate Azure AI Search Index

In this section we will:

1. **Create** an Azure AI Search index called `myfitnessindex` with a schema suited for fitness items
2. **Upload** sample documents containing fitness equipment data
3. **Verify** that the documents are searchable

Make sure your environment has the appropriate search credentials (typically obtained via your AI Foundry project).

In [1]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchFieldDataType, SearchableField
from azure.search.documents import SearchClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType

# Load environment variables
notebook_path = Path().absolute()
env_path = notebook_path.parent.parent / '.env'  # Adjust path as needed
load_dotenv(env_path)

connection_string = os.environ.get("PROJECT_CONNECTION_STRING")
if not connection_string:
  raise ValueError("🚨 PROJECT_CONNECTION_STRING not set in .env.")

# Initialize the AI Project client to access project resources
try:
  project_client = AIProjectClient(
      credential=DefaultAzureCredential(),
      endpoint=connection_string
  )
  print("✅ Initialized AIProjectClient")
except Exception as e:
  print(f"❌ Error initializing AIProjectClient: {e}")

# Get the Azure AI Search connection details from the project (including endpoint and API key)
search_conn = project_client.connections.get_default(
    connection_type=ConnectionType.AZURE_AI_SEARCH,
    include_credentials=True
)

if not search_conn:
  raise RuntimeError(
      "❌ No default Azure AI Search connection found in your project.")

# Define the index name for our fitness data
index_name = "myfitnessindex"

try:
  credential = AzureKeyCredential(search_conn.credentials["key"])
  index_client = SearchIndexClient(
      endpoint=search_conn.target, credential=credential)
  print("✅ Created SearchIndexClient")

  search_client = SearchClient(
      endpoint=search_conn.target,
      index_name=index_name,
      credential=credential
  )
  print("✅ Created SearchClient for document operations")
except Exception as e:
  print(f"❌ Error creating search clients: {e}")

✅ Initialized AIProjectClient
✅ Created SearchIndexClient
✅ Created SearchClient for document operations


### Define the Index Schema

We will create an index with the following fields:

- `FitnessItemID`: Unique key
- `Name`: Searchable text field (also filterable)
- `Category`: Searchable, filterable, and facetable (e.g. Strength, Cardio, Flexibility)
- `Price`: Numeric field (filterable, sortable, and facetable)
- `Description`: Full-text searchable field

In [2]:
def create_fitness_index():
  fields = [
      SimpleField(name="FitnessItemID",
                  type=SearchFieldDataType.String, key=True),
      SearchableField(
          name="Name", type=SearchFieldDataType.String, filterable=True),
      SearchableField(name="Category", type=SearchFieldDataType.String,
                      filterable=True, facetable=True),
      SimpleField(name="Price", type=SearchFieldDataType.Double,
                  filterable=True, sortable=True, facetable=True),
      SearchableField(name="Description", type=SearchFieldDataType.String)
  ]

  index = SearchIndex(name=index_name, fields=fields)

  # Delete the index if it already exists (for a fresh start)
  if index_name in [x.name for x in index_client.list_indexes()]:
    index_client.delete_index(index_name)
    print(f"🗑️ Deleted existing index: {index_name}")

  created = index_client.create_index(index)
  print(f"🎉 Created index: {created.name}")


# Create the index
create_fitness_index()

🗑️ Deleted existing index: myfitnessindex
🎉 Created index: myfitnessindex


### Upload Sample Documents

Now we’ll add some sample fitness items to `myfitnessindex`.

In [3]:
def upload_fitness_docs():
  search_client = SearchClient(
      endpoint=search_conn.target,
      index_name=index_name,
      credential=AzureKeyCredential(search_conn.credentials["key"])
  )

  sample_docs = [
      {
          "FitnessItemID": "1",
          "Name": "Adjustable Dumbbell",
          "Category": "Strength",
          "Price": 59.99,
          "Description": "A compact, adjustable weight for targeted muscle workouts."
      },
      {
          "FitnessItemID": "2",
          "Name": "Yoga Mat",
          "Category": "Flexibility",
          "Price": 25.0,
          "Description": "Non-slip mat designed for yoga, Pilates, and other exercises."
      },
      {
          "FitnessItemID": "3",
          "Name": "Treadmill",
          "Category": "Cardio",
          "Price": 499.0,
          "Description": "A sturdy treadmill with adjustable speed and incline settings."
      },
      {
          "FitnessItemID": "4",
          "Name": "Resistance Bands",
          "Category": "Strength",
          "Price": 15.0,
          "Description": "Set of colorful bands for light to moderate resistance workouts."
      }
  ]

  result = search_client.upload_documents(documents=sample_docs)
  print(f"🚀 Upload result: {result}")


upload_fitness_docs()
print("✅ Documents uploaded to search index")

🚀 Upload result: [<azure.search.documents._generated.models._models_py3.IndexingResult object at 0x7f0cf81fcbf0>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x7f0cf81fccb0>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x7f0cf81fcc50>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x7f0cf81fcce0>]
✅ Documents uploaded to search index


### Verify the Documents

Let’s perform a basic search query (e.g. for items in the **Strength** category) to ensure everything is working.

In [4]:
results = search_client.search(search_text="Strength", filter=None, top=10)

print("🔍 Search results for 'Strength':")
print("-" * 50)
found_items = False
for doc in results:
  found_items = True
  print(f"Name: {doc['Name']}")
  print(f"Category: {doc['Category']}")
  print(f"Price: ${doc['Price']:.2f}")
  print(f"Description: {doc['Description']}")
  print("-" * 50)

if not found_items:
  print("No matching items found.")

🔍 Search results for 'Strength':
--------------------------------------------------
Name: Resistance Bands
Category: Strength
Price: $15.00
Description: Set of colorful bands for light to moderate resistance workouts.
--------------------------------------------------
Name: Adjustable Dumbbell
Category: Strength
Price: $59.99
Description: A compact, adjustable weight for targeted muscle workouts.
--------------------------------------------------


## 2. Create Semantic Kernel Agent with Azure AI Search Tool

In this section we’ll use Semantic Kernel’s Azure AI Agent abstractions to build a fitness shopping assistant. This agent will:

- Use your Azure OpenAI model (specified by `MODEL_DEPLOYMENT_NAME`)
- Attach an **Azure AI Search tool** (pointing to `myfitnessindex`)
- Engage in an asynchronous conversation that queries the index based on user input

The code below uses asynchronous Python (with `asyncio`) and Semantic Kernel classes from the `semantic_kernel` package.

In [ ]:
import logging

from azure.ai.projects.aio import AIProjectClient
from azure.ai.agents.models import AzureAISearchTool
from azure.identity.aio import DefaultAzureCredential
from azure.ai.projects.models import ConnectionType

from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
import os

logging.basicConfig(level=logging.WARNING)

# For this demo, we will use the same index name as before
AZURE_AI_SEARCH_INDEX_NAME = "myfitnessindex"

# Get required environment variables
model_deployment_name = os.environ.get("MODEL_DEPLOYMENT_NAME")
project_connection_string = os.environ.get("PROJECT_CONNECTION_STRING")

if not model_deployment_name:
  raise ValueError("🚨 MODEL_DEPLOYMENT_NAME not set in .env")
if not project_connection_string:
  raise ValueError("🚨 PROJECT_CONNECTION_STRING not set in .env")

# Create agent settings with required parameters
ai_agent_settings = AzureAIAgentSettings(
    model_deployment_name=model_deployment_name,
    project_connection_string=project_connection_string
)

async with (
    DefaultAzureCredential() as creds,
    AIProjectClient(
        credential=creds,
        endpoint=project_connection_string
    ) as client,
):
  # List available connections and find one of type Azure AI Search
  conn_list = []
  async for conn in client.connections.list():
    conn_list.append(conn)
  ai_search_conn_id = ""
  for conn in conn_list:
    if conn.type == ConnectionType.AZURE_AI_SEARCH:
      ai_search_conn_id = conn.id
      break

  if not ai_search_conn_id:
    print("❌ No Azure AI Search connection found.")
    raise ValueError("❌ No Azure AI Search connection found.")

  # Create the Azure AI Search tool pointing to our fitness index
  ai_search_tool = AzureAISearchTool(
      index_connection_id=ai_search_conn_id,
      index_name=AZURE_AI_SEARCH_INDEX_NAME
  )

  # Create the agent definition with instructions for a fitness shopping assistant
  agent_definition = await client.agents.create_agent(
      model=ai_agent_settings.model_deployment_name,
      name="sk-health-search-agent"
      instructions="""
            You are a Fitness Shopping Assistant. You help users find fitness equipment based on their queries.
            Always include a disclaimer that you are not providing medical advice.
        """,
      tools=ai_search_tool.definitions,
      tool_resources=ai_search_tool.resources,
      headers={"x-ms-enable-preview": "true"},
  )

  # Create the Semantic Kernel Azure AI Agent
  agent = AzureAIAgent(
      client=client,
      definition=agent_definition,
  )

  # 3. Create a thread for the agent
  # If no thread is provided, a new thread will be
  # created and returned with the initial response
  thread: AzureAIAgentThread = None

  # Define some example fitness queries
  user_queries = [
      "Which items are best for strength training?",
      "I need something for cardio under $300. Any suggestions?"
  ]

  try:
    for query in user_queries:
      print(f"\n# User: {query}\n")
      response = await agent.get_response(messages=query, thread=thread)
      print(f"# {response.name}: {response}")
  finally:
    # Clean up the conversation thread and agent
    await thread.delete() if thread else None
    await client.agents.delete_agent(agent.id)
    print("🗑️ Cleaned up agent and thread")


# User: Which items are best for strength training?

# azure_agent_zEIFXfJs: For strength training, the best equipment typically includes:

1. Free Weights: Dumbbells, Barbells, Kettlebells — great for versatile strength exercises.
2. Weight Machines: Target specific muscle groups safely with guided movement.
3. Resistance Bands: Portable and effective for adding resistance.
4. Adjustable Benches: Essential for various weightlifting exercises.
5. Power Racks or Squat Racks: Important for heavy lifting and safety.
6. Medicine Balls and Slam Balls: Useful for dynamic strength workouts.
7. Pull-Up Bars: Great for upper body strength.
8. Sandbags: Versatile for functional strength training.

If you have particular preferences or goals, I can help narrow down options. Also, note that I am not providing medical advice, so please consult a fitness professional to ensure the equipment suits your personal needs.

# User: I need something for cardio under $300. Any suggestions?

# azure_agent_z

## 3. Cleanup

For this demo we already clean up the agent and thread inside the async function. In case you want to remove the search index as well (for a fresh start), run the cell below.

In [6]:
try:
  index_client.delete_index(index_name)
  print(f"🗑️ Deleted index {index_name}")
except Exception as e:
  print(f"Error deleting index: {e}")

🗑️ Deleted index myfitnessindex


# 🎉 Congrats!

You've successfully:

1. Created an Azure AI Search index and populated it with fitness data
2. Verified the data via a basic search query
3. Built and run a Semantic Kernel Agent that leverages Azure AI Search to answer natural language queries

Feel free to explore further enhancements (e.g. integrating more tools or advanced evaluation) and enjoy your journey with Azure AI Foundry and Semantic Kernel!

In [ ]:
# Copyright (c) Microsoft. All rights reserved.

import asyncio

from azure.identity.aio import DefaultAzureCredential

from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

"""
The following sample demonstrates how to create an Azure AI agent that answers
user questions. This sample demonstrates the basic steps to create an agent
and simulate a conversation with the agent.

The interaction with the agent is via the `get_response` method, which sends a
user input to the agent and receives a response from the agent. The conversation
history is maintained by the agent service, i.e. the responses are automatically
associated with the thread. Therefore, client code does not need to maintain the
conversation history.
"""


# Simulate a conversation with the agent
USER_INPUTS = [
    "Hello, I am John Doe.",
    "What is your name?",
    "What is my name?",
]


async def main() -> None:
  async with (
      DefaultAzureCredential() as creds,
      AzureAIAgent.create_client(credential=creds, endpoint=os.environ["PROJECT_CONNECTION_STRING"]) as client,
  ):
    # 1. Create an agent on the Azure AI agent service
    agent_definition = await client.agents.create_agent(
        model="gpt-4.1-nano",
        name="Assistant",
        instructions="Answer the user's questions.",
    )

    # 2. Create a Semantic Kernel agent for the Azure AI agent
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
    )

    # 3. Create a thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread = None

    try:
      for user_input in USER_INPUTS:
        print(f"# User: {user_input}")
        # 4. Invoke the agent with the specified message for response
        response = await agent.get_response(messages=user_input, thread=thread)
        print(f"# {response.name}: {response}")
        thread = response.thread
    finally:
      # 6. Cleanup: Delete the thread and agent
      await thread.delete() if thread else None
      await client.agents.delete_agent(agent.id)

    """
        Sample Output:
        # User: Hello, I am John Doe.
        # Assistant: Hello, John! How can I assist you today?
        # User: What is your name?
        # Assistant: I'm here as your assistant, so you can just call me Assistant. How can I help you today?
        # User: What is my name?
        # Assistant: Your name is John Doe. How can I assist you today, John?
        """


await main()

# User: Hello, I am John Doe.
# Assistant: Hello, John! How can I assist you today?
# User: What is your name?
# Assistant: I'm an AI language model, so I don't have a personal name, but you can call me ChatGPT. How can I help you today?
# User: What is my name?
# Assistant: You mentioned your name is John Doe. How can I assist you today, John?
